In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 16)

In [2]:
import pandas as pd

In [3]:
IN_PATH: str = '../results/manifolds/raw'
OUT_PATH: str = '../results/manifolds'

In [4]:
DATASETS: list = ['train', 'test']

In [5]:
MODELS: list = [ 'base', 'textattack', 'fabriceyhc', 'wakaka']

In [6]:
COLS_OF_INTEREST: list = ['mean', 'std', 'min', 'max']

In [7]:
### Load Datasets into memory

In [8]:
def convert_to_multi_col(df:pd.DataFrame, meta_col: str) -> pd.DataFrame:
    df.columns = pd.MultiIndex.from_product([[meta_col], df.columns], names=['model', 'metrics'])

    return df

In [9]:
analysis: dict = {
    data: pd.concat(
        [(
            pd
            .read_csv(f'{IN_PATH}/{data}.{model}.csv', index_col=0)
            .set_index(['dim'])
            .pipe(convert_to_multi_col, model)
        )
            for model in MODELS],
        axis=1)
    for data in DATASETS
}

In [10]:
for label, data in analysis.items():
    display(label, data)

'train'

model                  base                                        textattack  \
metrics dispersion_positive dispersion_negative  distance dispersion_positive   
dim                                                                             
768                6.138071            5.764313  1.259246            7.117969   
576                6.139669            5.765659  1.194376            7.116583   
384                6.139204            5.765113  1.199099            7.115844   
192                6.137501            5.763306  1.219064            7.113526   
96                 6.134574            5.760337  1.241529            7.108215   
48                 6.128319            5.753036  1.305715            7.097139   
24                 6.114873            5.737919  1.420860            7.072455   
12                 6.085098            5.701169  1.636281            7.016590   
6                  6.018489            5.616422  1.876313            6.867037   
3                  5.918405            5.411482  1.779940            6.329559   

model                                          fabriceyhc                      \
metrics dispersion_negative  distance dispersion_positive dispersion_negative   
dim                                                                             
768                6.738079  7.820493            6.040889            7.644175   
576                6.736554  7.822101            6.039529            7.641622   
384                6.735819  7.823868            6.039256            7.641588   
192                6.733359  7.829962            6.038501            7.641333   
96                 6.727870  7.843417            6.037833            7.640487   
48                 6.716420  7.870813            6.035165            7.639025   
24                 6.690959  7.932147            6.027100            7.631796   
12                 6.632266  8.064154            6.010790            7.616623   
6                  6.482053  8.374858            5.947494            7.559823   
3                  5.987389  9.175378            5.612881            7.313653   

model                           wakaka                                 
metrics   distance dispersion_positive dispersion_negative   distance  
dim                                                                    
768      22.864886            8.561553            7.437458  11.483087  
576      22.865088            8.559432            7.435664  11.485481  
384      22.865183            8.559021            7.435237  11.485998  
192      22.865379            8.557336            7.433518  11.488414  
96       22.865782            8.553857            7.429942  11.494228  
48       22.866592            8.544770            7.421367  11.507466  
24       22.869685            8.525990            7.402648  11.537118  
12       22.875936            8.479190            7.356498  11.609513  
6        22.900114            8.340063            7.223993  11.823141  
3        23.043346            7.904102            6.753563  12.400316

'test'

model                  base                                        textattack  \
metrics dispersion_positive dispersion_negative  distance dispersion_positive   
dim                                                                             
768                6.084546            5.798742  1.229414            7.183807   
576                6.086450            5.800295  1.157035            7.182440   
384                6.086301            5.800096  1.155812            7.181604   
192                6.084319            5.798407  1.178628            7.179190   
96                 6.082008            5.795524  1.204912            7.173834   
48                 6.074534            5.788623  1.268671            7.162849   
24                 6.058615            5.772423  1.404744            7.136895   
12                 6.031680            5.745048  1.566630            7.078968   
6                  5.954286            5.664161  1.905443            6.925149   
3                  5.892543            5.513398  1.143659            6.395235   

model                                          fabriceyhc                      \
metrics dispersion_negative  distance dispersion_positive dispersion_negative   
dim                                                                             
768                6.935167  7.312816            7.438157            9.273344   
576                6.933758  7.314245            7.436355            9.271827   
384                6.933000  7.316373            7.436274            9.271746   
192                6.930747  7.322635            7.435166            9.271504   
96                 6.926006  7.335768            7.434171            9.270402   
48                 6.916276  7.361726            7.431265            9.268296   
24                 6.892173  7.425125            7.426020            9.265023   
12                 6.839176  7.558527            7.405454            9.254471   
6                  6.697920  7.877356            7.336097            9.197858   
3                  6.210816  8.715597            6.961802            8.920493   

model                           wakaka                                 
metrics   distance dispersion_positive dispersion_negative   distance  
dim                                                                    
768      19.964325            8.446871            7.581062  11.398275  
576      19.964597            8.444721            7.579173  11.400781  
384      19.964672            8.444202            7.578564  11.401899  
192      19.965052            8.442202            7.576835  11.405012  
96       19.965542            8.438007            7.573840  11.410919  
48       19.966721            8.429272            7.565218  11.424533  
24       19.969651            8.408910            7.547488  11.455621  
12       19.978131            8.357105            7.501981  11.534418  
6        20.011267            8.207643            7.370401  11.754118  
3        20.250669            7.730735            6.907276  12.368755

In [11]:
### Calculate Centroid Distance and Cluster Dispersion

In [12]:
distances: dict = {
    label: (
        data
        .filter(regex=".*extra_distance")
        .describe()
        .T
    )
    for label, data in analysis.items()
}

In [13]:
for label, data in distances.items():
    formatted: pd.DataFrame = (
        data
        [COLS_OF_INTEREST]
        .round(3)
    )
    formatted.to_csv(f'{OUT_PATH}/{label}.metric.distance.csv')
    formatted.style.to_latex(f'{OUT_PATH}/{label}.metric.distance.tex')
    display(label, formatted)
    # print(formatted.to_markdown())

'train'

,,mean,std,min,max
model,metrics,,,,
base,distance,1.413,0.257,1.194,1.876
textattack,distance,8.056,0.430,7.820,9.175
fabriceyhc,distance,22.888,0.056,22.865,23.043
wakaka,distance,11.631,0.290,11.483,12.400


'test'

,,mean,std,min,max
model,metrics,,,,
base,distance,1.321,0.245,1.144,1.905
textattack,distance,7.554,0.445,7.313,8.716
fabriceyhc,distance,20.000,0.089,19.964,20.251
wakaka,distance,11.555,0.306,11.398,12.369


In [14]:
dispersion: dict = {
    label: (
        data
        .filter(regex=".*intra_distance")
        .describe()
        .T
    )
    for label, data in analysis.items()
}

In [15]:
for label, data in dispersion.items():
    formatted: pd.DataFrame = (
        data
        [['mean', 'std', 'min', 'max']]
        .round(3)
    )
    formatted.to_csv(f'{OUT_PATH}/{label}.metric.dispersion.csv')
    formatted.style.to_latex(f'{OUT_PATH}/{label}.metric.dispersion.tex')
    display(label, formatted)
    # print(formatted.to_markdown())

'train'

mean    std    min    max
model      metrics                                        
base       dispersion_positive  6.095  0.073  5.918  6.140
           dispersion_negative  5.704  0.113  5.411  5.766
textattack dispersion_positive  6.995  0.247  6.330  7.118
           dispersion_negative  6.618  0.235  5.987  6.738
fabriceyhc dispersion_positive  5.983  0.133  5.613  6.041
           dispersion_negative  7.597  0.103  7.314  7.644
wakaka     dispersion_positive  8.459  0.206  7.904  8.562
           dispersion_negative  7.333  0.214  6.754  7.437

'test'

mean    std    min    max
model      metrics                                        
base       dispersion_positive  6.044  0.067  5.893  6.086
           dispersion_negative  5.748  0.093  5.513  5.800
textattack dispersion_positive  7.060  0.247  6.395  7.184
           dispersion_negative  6.822  0.227  6.211  6.935
fabriceyhc dispersion_positive  7.374  0.148  6.962  7.438
           dispersion_negative  9.226  0.110  8.920  9.273
wakaka     dispersion_positive  8.335  0.225  7.731  8.447
           dispersion_negative  7.478  0.211  6.907  7.581